In [128]:
import csv 
import os

# Create CSV Files

In [129]:
def convert_to_mib(memory):
    # Convert memory to MiB if necessary
    if 'GiB' in memory:
        memory_in_gib = float(memory[:-3])
        memory_in_mib = memory_in_gib * 1024
        return f"{memory_in_mib:.2f}MiB"
    elif 'MiB' in memory:
        return memory
    else:
        raise ValueError("Invalid memory format")


def extract_info(file_name):
    cpu_percentages = []
    mem_usages = []

    with open(file_name, 'r') as file:
        for line in file:
            values = line.split()
            
            cpu_percentages.append(float(values[2][:-1]))
            mem_usages.append(float(convert_to_mib(values[3])[:-3]))

    return cpu_percentages, mem_usages


def write_to_csv(cpu_percentages, mem_usages, output_file):
    with open(output_file, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['CPU (%)', 'Memory Usage (MB)'])
        for i in range(len(cpu_percentages)):
            writer.writerow([cpu_percentages[i], mem_usages[i]])

In [130]:
def list_txt_files_in_folder(folder_path):
    txt_files = []
    try:
        for file in os.listdir(folder_path):
            if file.endswith(".txt") and os.path.isfile(os.path.join(folder_path, file)):
                txt_files.append(file)
    except Exception as e:
        print(f"Error occurred while listing .txt files: {e}")
    return txt_files

In [131]:
input_folder_path = "./txt/"
output_folder_path = "./output/"

files_in_folder = list_txt_files_in_folder(input_folder_path)

for file_name in files_in_folder:
    print(file_name)
    file_path = input_folder_path + file_name
    output_file_path = "./output/" + file_name.split()[0] + ".csv"

    cpu_percentages, mem_usages = extract_info(file_path)
    write_to_csv(cpu_percentages, mem_usages, output_file_path)
    
    print(f'Data has been written to {output_file_path}')

data_centralized.txt
Data has been written to ./output/data_centralized.txt.csv
data_distribute.txt
Data has been written to ./output/data_distribute.txt.csv


# Generate Charts